<h1>Implementação - Modificação do projeto "Processamento Digital de Imagens Aplicado a Identificação Automática de Placas de Trânsito"</h1>

Esta é a modificação proposta para a implementação do projeto "Processamento Digital de Imagens Aplicado a Identificação Automática de Placas de Trânsito", disponível em: http://ojs.unirg.edu.br/index.php/1/article/view/2280

O artigo foi utilizado como referência para o desenvolvimento deste projeto.

In [1]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.pyplot as pp
import matplotlib.patches as patches
import numpy as np
import cv2
import os
import math

from collections import deque
from skimage import data
from skimage.color import rgb2hsv
from skimage import filters
from skimage import exposure
from skimage import color, img_as_float
from skimage import measure

<h1>Primeira etapa</h1>

Ao  início  do  algoritmo,  é  lida  a  imagem  de  entrada  (no  espaço  de  cores  padrão RGB), em que se quer verificar a existência de uma placa de parada obrigatória.

In [2]:
path = os.path.join(os.getcwd(), 'Dataset-modificado')
print(path)

C:\Users\anaca\Documents\IFCE\PDI\Trabalho final\Dataset-modificado


In [3]:
data_set = []
gray_data_set = []
        
for caminho, d, file in os.walk(path):
    for filename in file:
        
        img = cv2.imread(os.path.join(caminho,filename))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        data_set.append(img)
        gray_data_set.append(gray)

<h1>Segunda etapa</h1>

Para  a  detecção  de  uma  placa  de  pare,  primeiramente  é  feita  a  separação  dos objetos  em  vermelho  do  restante  da  imagem. Esta  tarefa  é  realizada  convertendo  a imagem  para  o  espaço  de  cores  HSV (função rgb2hsv). Busca-se  por pixels com valores de matiz (hue) 10% mais próximos às extremidades iniciais e finais do conjunto possível de valores representados nesta banda (a matiz armazena as variações cromáticas  da  imagem  de  entrada  com  valores  compreendidos  entre  0  e 1) e com  um valor  de  saturação acima  de 50% (pois procura-se por cores mais puras). Isto  gera  uma máscara que é aplicada à imagem original separando os objetos com cor predominantemente vermelha dos demais objetos presentes na  imagem.

In [4]:
def red_detection(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    ## Gen lower mask (0-5) and upper mask (175-180) of RED
    mask1 = cv2.inRange(img_hsv, (0,50,20), (5,255,255))
    mask2 = cv2.inRange(img_hsv, (175,50,20), (180,255,255))

    ## Merge the mask and crop the red regions
    mask = cv2.bitwise_or(mask1, mask2 )
    croped = cv2.bitwise_and(img, img, mask=mask)
    
    return mask, croped

Para finalizar, somente são considerados objetos com dimensão de, no mínimo, 10% do tamanho total da imagem. Qualquer objeto detectado, com dimensão inferior a 10% da dimensão da imagem de entrada, é descartado pela chamada da função bwareaopen.

In [5]:
def dimension10(mask,img):
    blur = cv2.blur(mask,(4,4))
    mask = cv2.threshold(blur, 175 , 250, cv2.THRESH_BINARY)
    mask = mask[1]
    croped = cv2.bitwise_and(img, img, mask=mask)
    
    return mask, croped

<h1>Terceira Etapa</h1>

Após segmentar um objeto vermelho da imagem, onde uma possível placa de pare estará,  a  imagem  passa  por  um  processo  de detecção  de  bordas  pela  máscara  de gradiente  binário  nesta  imagem  segmentada,  utilizando  o  filtro  de  Sobel  aplicado  com  o auxílio de funções do software.

In [6]:
def edge(croped):
    edges = cv2.Canny(croped,100,200)
    
    return edges

Depois  da  aplicação  do  filtro  de Sobel,  é  possível  que  bordas  do mesmo  objeto fiquem  desconectadas.  Então  a  imagem  gerada  é dilatada  para  realizar  a  junção  de bordas de um mesmo objeto. Essa dilatação é feita através da função imdilate do Matlab.

In [7]:
def sobel(edges):
    kernel = np.ones((3,3),np.uint8)
    dilate = cv2.dilate(edges,kernel,iterations = 1)
    
    return dilate

<h1>Quarta Etapa</h1>

Como  o  propósito  do  algoritmo  é  detectar  apenas  as  bordas  externas  de  uma possível  placa,  ao  resultado  do  processo  anterior,  é  aplicado outro  operador morfológico para  preencher  o  interior de  objetos conectados  (função imfill), resultando  em  uma imagem  onde todos  os objetos  com  bordas  externas  conectadas  estão  totalmente preenchidos.

In [8]:
def get_seed(image):
    w, h = image.shape
    semente = np.zeros((w, h))
    semente[:, :3] = 255 # Lateral esquerda
    semente[:, (h-3):] = 255 # Lateral direita
    
    return semente

In [9]:
def neighborhood(x, y, w, h):
    lista = deque()
    
    pontos = [(x-1,y), (x+1, y), (x,y-1), (x,y+1),
              (x-1,y+1), (x+1, y+1), (x-1,y-1), (x+1,y-1),
             ]
    for p in pontos:
        if (p[0]>=0 and p[1]>=0 and p[0]<w and p[1]<h):
            lista.append((p[0], p[1]))
            
    return lista

In [10]:
def grow_region(image, reg, epsilon=5):
    w, h = image.shape
    
    fila = deque()
    for x in range(w):
        for y in range(h):
            if reg[x,y]==255:
                fila.append((x,y))
                
    while fila: 
        ponto = fila.popleft()
        x = ponto[0]
        y = ponto[1]

        v_list = neighborhood(x, y, w, h)
        for v in v_list:
            v_x = v[0]
            v_y = v[1]
            if( (reg[v_x][v_y]!=255) and (abs(image[x][y]-image[v_x][v_y])<epsilon)):
                reg[v_x][v_y] = 255
                fila.append((v_x,v_y))
    return reg

In [11]:
def close(le):
    kernel = np.ones((4,4),np.uint8)
    closing = cv2.morphologyEx(le, cv2.MORPH_CLOSE, kernel)
    cv2.imwrite('closing.jpg',closing)
    return closing

Posteriormente,  para  melhorar  a  definição  geométrica  do  objeto  detectado  na imagem, é realizada uma erosão com o auxílio da função imerode, cujo resultado é uma imagem mais bem definida do objeto de interesse.

<h1>Quinta Etapa</h1>

Neste  ponto,  espera-se  ter  uma  imagem  binária  com  a  geometria  bem definida  do objeto de interesse (um objeto relativamente grande, na cor vermelha, que não esteja em contato com qualquer extremidade da imagem). Esta imagem é utilizada como entrada no algoritmo de detecção de retas pela transformada de Hough.
Neste algoritmo, o primeiro passo  é  detectar  as  bordas  do  objeto  de entrada  e,  para  isto,  é  utilizado  o  detector  de bordas ótimo de Canny.


In [16]:
def cannyClose(closing):
    copyClose = np.uint8(closing)
    edges = cv2.Canny(copyClose,10,20)

    return edges

Em vez de utilizar a Transformação de Hough, foi utilizado detecção de contornos a partir da borda da segmentação da placa. Para finalizar, é plotado um retângulo ao redor da placa observada, tendo-se como referência  os  pontos  extremos  que  delimitam  (acima,  abaixo,  à esquerda  e  à  direita)  a região provável de localização da placa na imagem de entrada.

As imagens com as placas detectadas são salvas em uma nova pasta.

In [15]:
def drawContourn(edges,img,a):
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print("Number of Contours found = " + str(len(contours)))
    cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
    for contour in contours:
        # Find bounding rectangles
        x,y,w,h = cv2.boundingRect(contour)
        
        # Draw the rectangle
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)

    path = os.path.join(os.getcwd(), 'Novas-imagens')
    cv2.imwrite(os.path.join(path,'contourn'+str(a)+'.jpg'),img)

In [14]:
from tqdm import tqdm

a = 1

for img in tqdm(data_set):    
    mask, croped = red_detection(img)
    mask, croped = dimension10(mask,img)
    edges = edge(croped)
    dilate = sobel(edges)
    
    semente = get_seed(dilate)
    le = grow_region(dilate, semente, epsilon=5)
    
    closing = close(le)
    edges = cannyClose(closing)
    drawContourn(edges, img, a)
    
    a += 1

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]<ipython-input-10-46c31f6ee4df>:19: RuntimeWarning: overflow encountered in ubyte_scalars
  if( (reg[v_x][v_y]!=255) and (abs(image[x][y]-image[v_x][v_y])<epsilon)):
  2%|█▋                                                                                 | 1/50 [00:05<04:23,  5.38s/it]

Number of Contours found = 1


  4%|███▎                                                                               | 2/50 [00:12<04:40,  5.85s/it]

Number of Contours found = 4


  6%|████▉                                                                              | 3/50 [00:19<04:55,  6.28s/it]

...

Number of Contours found = 8
Number of Contours found = 13


 46%|█████████████████████████████████████▋                                            | 23/50 [05:13<05:58, 13.28s/it]

Number of Contours found = 0


 46%|█████████████████████████████████████▋                                            | 23/50 [05:14<06:09, 13.68s/it]


KeyboardInterrupt: 